# Comparison of CMC-learners and other approaches (simulations)

**On Nie and Wager (2021) synthetic data**

In [1]:
# import everyting from data module which is located ../../src/data.py
import sys
import numpy as np
sys.path.append('../..')
from src.datasets.r_learner_datasets import (simulate_nuisance_and_easy_treatment,
                                             simulate_randomized_trial,
                                             simulate_easy_propensity_difficult_baseline,
                                             simulate_unrelated_treatment_control)

In [2]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

get_save_data = True
heteroscedastic_epsilon = True
NSim = 100
alphas = [0.05, 0.1, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95]
learner = RandomForestRegressor
learner_name = "RF"
MC_samples = 100
adaptive_conformal = False
if adaptive_conformal:
    adaptive_conformal_name = "adaptive"
else:
    adaptive_conformal_name = "nonadaptive"

In [3]:
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from src.mc_conformal_metalearner.mc_conformal_metalearners import Conformal_MC_S_Learner, Conformal_MC_T_Learner, Conformal_MC_X_Learner
from src.conformal_metalearners.CM_learner import CM_learner
from tqdm import tqdm

from src.weighted_conformal_prediction.wcp import NaiveWCP, NestedWCP


for i, setup in enumerate([simulate_nuisance_and_easy_treatment, simulate_randomized_trial, simulate_easy_propensity_difficult_baseline, simulate_unrelated_treatment_control]):
    if i == 0:
        setup_name = "A"
    elif i == 1:
        setup_name = "B"
    elif i == 2:
        setup_name = "C"
    else:
        setup_name = "D"
    if heteroscedastic_epsilon:
        path_train = f"../../data/simulations/r_learner/setup{setup_name}/simulations_{setup_name}_train_heteroscedastic.csv"
        path_test = f"../../data/simulations/r_learner/setup{setup_name}/simulations_{setup_name}_test_heteroscedastic.csv"
    else:
        path_train = f"../../data/simulations/r_learner/setup{setup_name}/simulations_{setup_name}_train.csv"
        path_test = f"../../data/simulations/r_learner/setup{setup_name}/simulations_{setup_name}_test.csv"
    for n in tqdm(range(NSim)):
        if get_save_data:
            ds_train = pd.read_csv(path_train)
            ds_test = pd.read_csv(path_train)
            y_train = ds_train["y"].values
            y_test = ds_test["y"].values
            X_train = ds_train[[f"X{i}" for i in range(5)]].values
            X_test = ds_test[[f"X{i}" for i in range(5)]].values
            W_train = ds_train["W"].values
            W_test = ds_test["W"].values
            tau_train = ds_train["tau"].values
            tau_test = ds_test["tau"].values
            b_train = ds_train["b"].values
            b_test = ds_test["b"].values
            ps_train = ds_train["ps"].values
            ps_test = ds_test["ps"].values
            y0_train = ds_train["y0"].values
            y0_test = ds_test["y0"].values
            y1_train = ds_train["y1"].values
            y1_test = ds_test["y1"].values
            ite_train = ds_train["ite"].values
            ite_test = ds_test["ite"].values
        else:
            y, X, treatment, tau, b, e, y0, y1 = setup(n=5000, c=0.0, heteroscedastic=heteroscedastic_epsilon)
            ite = y1 - y0
            ps = e
            (y_train, y_test, X_train, X_test,
            W_train, W_test,
            tau_train, tau_test, b_train, b_test,
            ps_train, ps_test, y0_train, y0_test,
            y1_train, y1_test, ite_train, ite_test) = train_test_split(y, X, treatment, tau, b, ps, y0, y1, ite,
                                                                        test_size=0.5)
            df_X_train = pd.DataFrame(X_train, columns=[f"X{i}" for i in range(X_train.shape[1])])
            df_X_test = pd.DataFrame(X_test, columns=[f"X{i}" for i in range(X_test.shape[1])])
            ds_train = pd.concat((df_X_train, pd.DataFrame({
                "y": y_train,
                "W": W_train,
                "tau": tau_train,
                "b": b_train,
                "ps": ps_train,
                "y0": y0_train,
                "y1": y1_train,
                "ite": ite_train
            })), axis=1)
            ds_test = pd.concat((df_X_test, pd.DataFrame({
                "y": y_test,
                "W": W_test,
                "tau": tau_test,
                "b": b_test,
                "ps": ps_test,
                "y0": y0_test,
                "y1": y1_test,
                "ite": ite_test
            })), axis=1)
            ds_train.to_csv(path_train)
            ds_test.to_csv(path_test)


        # Initialize the learner
        conformal_pseudo_MC_T_Learner = Conformal_MC_T_Learner(
            learner(),
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
        )

        conformal_MC_T_Learner = Conformal_MC_T_Learner(
            learner(),
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
        )

        conformal_pseudo_MC_S_Learner = Conformal_MC_S_Learner(
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
        )

        conformal_MC_S_Learner = Conformal_MC_S_Learner(
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
        )
        conformal_pseudo_MC_X_Learner = Conformal_MC_X_Learner(
            learner(),
            learner(),
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
        )
        # Fit the learner
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train)
        conformal_MC_X_Learner = Conformal_MC_X_Learner(
            learner(),
            learner(),
            learner(),
            adaptive_conformal=adaptive_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
        )
        # Fit the learner
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            conformal_MC_X_Learner.fit(X_train, y_train, W_train)
        naive_WCP = NaiveWCP(
                        learner(),
                        learner(),
                        adaptive_conformal=adaptive_conformal)

        # Fit the learner
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train)
        conformal_MC_T_Learner.fit(X_train, y_train, W_train)
        naive_WCP.fit(X_train, y_train, W_train, ps_train)
        with warnings.catch_warnings(action="ignore", category=UserWarning):
            # Suppress warning that is thrown saying that calibration example is too small
            # However, this is a bug in the crepes library in this case
            conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train)
            conformal_MC_S_Learner.fit(X_train, y_train, W_train)


        int_results_y0 = []
        int_results_y1 = []
        int_results_pseudo_MC_T_ite = []
        int_results_MC_T_ite = []
        int_results_pseudo_MC_S_ite = []
        int_results_MC_S_ite = []
        int_results_pseudo_MC_X_ite = []
        int_results_MC_X_ite = []
        int_results_CM_ite = []
        int_results_naive_ite = []
        int_results_exact_ite = []
        int_results_inexact_ite = []

        pred_results_y0 = []
        pred_results_y1 = []
        pred_results_pseudo_MC_T_ite = []
        pred_results_MC_T_ite = []
        pred_results_pseudo_MC_S_ite = []
        pred_results_MC_S_ite = []
        pred_results_pseudo_MC_X_ite = []
        pred_results_MC_X_ite = []
        pred_results_CM_ite = []
        pred_results_naive_ite = []
        pred_results_exact_ite = []
        pred_results_inexact_ite = []

        pred_results_pseudo_MC_T_ite_conformal_mean = []
        pred_results_MC_T_ite_conformal_mean = []
        pred_results_pseudo_MC_S_ite_conformal_mean = []
        pred_results_MC_S_ite_conformal_mean = []
        pred_results_pseudo_MC_X_ite_conformal_mean = []
        pred_results_MC_X_ite_conformal_mean = []

        coverage_y0 = []
        coverage_y1 = []
        coverage_pseudo_MC_T_ite = []
        coverage_MC_T_ite = []
        coverage_pseudo_MC_S_ite = []
        coverage_MC_S_ite = []
        coverage_pseudo_MC_X_ite = []
        coverage_MC_X_ite = []
        coverage_CM_ite = []
        coverage_naive_ite = []
        coverage_exact_ite = []
        coverage_inexact_ite = []

        int_width_y0 = []
        int_width_y1 = []
        int_width_pseudo_MC_T = []
        int_width_MC_T = []
        int_width_pseudo_MC_S = []
        int_width_MC_S = []
        int_width_pseudo_MC_X = []
        int_width_MC_X = []
        int_width_CM = []
        int_width_naive_ite = []
        int_width_exact_ite = []
        int_width_inexact_ite = []

        rmse_y0 = []
        rmse_y1 = []
        rmse_pseudo_MC_T_ite, rmse_pseudo_MC_T_ite_conformal_mean = [], []
        rmse_MC_T_ite,  rmse_MC_T_ite_conformal_mean = [], []
        rmse_pseudo_MC_S_ite, rmse_pseudo_MC_S_ite_conformal_mean = [], []
        rmse_MC_S_ite, rmse_MC_S_ite_conformal_mean = [], []
        rmse_pseudo_MC_X_ite, rmse_pseudo_MC_X_ite_conformal_mean = [], []
        rmse_MC_X_ite, rmse_MC_X_ite_conformal_mean = [], []
        rmse_CM_ite = []
        rmse_naive_ite = []
        rmse_exact_ite = []
        rmse_inexact_ite = []


        for alpha in alphas:
            int_y0_test = conformal_MC_T_Learner.predict_int_y0(X_test, confidence=1-alpha)
            int_y1_test = conformal_MC_T_Learner.predict_int_y1(X_test, confidence=1-alpha)
            int_pseudo_MC_ite_test = conformal_pseudo_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_ite_test = conformal_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_S_ite_test = conformal_pseudo_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_S_ite_test = conformal_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_X_ite_test = conformal_pseudo_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_X_ite_test = conformal_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            conformal_Learner = CM_learner(metalearner="DR", alpha=alpha)
            conformal_Learner.fit(X_train, y_train, W_train, ps_train)
            int_CM_ite_test = conformal_Learner.predict_int(X_test)
            int_naive_ite_test = naive_WCP.predict_int(X_test, ps_test, confidence=1-alpha)
            exact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            learner(),
                            adaptive_conformal=adaptive_conformal,
                            exact=True)
            exact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_exact_ite_test = exact_wcp.predict_int(X_test, ps_test)
            inexact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            adaptive_conformal=adaptive_conformal,
                            exact=False)
            inexact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_inexact_ite_test = inexact_wcp.predict_int(X_test, ps_test)

            int_results_y0.append(int_y0_test)
            int_results_y1.append(int_y1_test)
            int_results_pseudo_MC_T_ite.append(int_pseudo_MC_ite_test)
            int_results_MC_T_ite.append(int_MC_ite_test)
            int_results_pseudo_MC_S_ite.append(int_pseudo_MC_S_ite_test)
            int_results_MC_S_ite.append(int_MC_S_ite_test)
            int_results_pseudo_MC_X_ite.append(int_pseudo_MC_X_ite_test)
            int_results_MC_X_ite.append(int_MC_X_ite_test)
            int_results_CM_ite.append(int_CM_ite_test)
            int_results_naive_ite.append(int_naive_ite_test)
            int_results_exact_ite.append(int_exact_ite_test)
            int_results_inexact_ite.append(int_inexact_ite_test)

            pred_results_y0.append(conformal_MC_T_Learner.predict_y0(X_test))
            pred_results_y1.append(conformal_MC_T_Learner.predict_y1(X_test))
            pred_results_pseudo_MC_T_ite.append(conformal_pseudo_MC_T_Learner.predict(X_test))
            pred_results_MC_T_ite.append(conformal_MC_T_Learner.predict(X_test))
            pred_results_pseudo_MC_S_ite.append(conformal_pseudo_MC_S_Learner.predict(X_test))
            pred_results_MC_S_ite.append(conformal_MC_S_Learner.predict(X_test))
            pred_results_pseudo_MC_X_ite.append(conformal_pseudo_MC_X_Learner.predict(X_test))
            pred_results_MC_X_ite.append(conformal_MC_X_Learner.predict(X_test))
            pred_results_CM_ite.append(conformal_Learner.predict(X_test))
            pred_results_naive_ite.append(naive_WCP.predict(X_test))
            pred_results_exact_ite.append(exact_wcp.predict(X_test))
            pred_results_inexact_ite.append(inexact_wcp.predict(X_test))

            pred_results_pseudo_MC_T_ite_conformal_mean.append(conformal_pseudo_MC_T_Learner.predict(X_test, conformal_mean=True))
            pred_results_MC_T_ite_conformal_mean.append(conformal_MC_T_Learner.predict(X_test, conformal_mean=True))
            pred_results_pseudo_MC_S_ite_conformal_mean.append(conformal_pseudo_MC_S_Learner.predict(X_test, conformal_mean=True))
            pred_results_MC_S_ite_conformal_mean.append(conformal_MC_S_Learner.predict(X_test, conformal_mean=True))
            pred_results_pseudo_MC_X_ite_conformal_mean.append(conformal_pseudo_MC_X_Learner.predict(X_test, conformal_mean=True))
            pred_results_MC_X_ite_conformal_mean.append(conformal_MC_X_Learner.predict(X_test, conformal_mean=True))

            coverage_y0.append(np.mean((int_y0_test[:, 0] < y0_test) &  (int_y0_test[:, 1] > y0_test)))
            coverage_y1.append(np.mean((int_y1_test[:, 0] < y1_test) &  (int_y1_test[:, 1] > y1_test)))
            coverage_pseudo_MC_T_ite.append(np.mean((int_pseudo_MC_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_ite_test[:, 1] > ite_test)))
            coverage_MC_T_ite.append(np.mean((int_MC_ite_test[:, 0] < ite_test) &  (int_MC_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_S_ite.append(np.mean((int_pseudo_MC_S_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_S_ite_test[:, 1] > ite_test)))
            coverage_MC_S_ite.append(np.mean((int_MC_S_ite_test[:, 0] < ite_test) &  (int_MC_S_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_X_ite.append(np.mean((int_pseudo_MC_X_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_X_ite_test[:, 1] > ite_test)))
            coverage_MC_X_ite.append(np.mean((int_MC_X_ite_test[:, 0] < ite_test) &  (int_MC_X_ite_test[:, 1] > ite_test)))
            coverage_CM_ite.append(np.mean((int_CM_ite_test[:, 0] < ite_test) &  (int_CM_ite_test[:, 1] > ite_test)))
            coverage_naive_ite.append(np.mean((int_naive_ite_test[:, 0] < ite_test) &  (int_naive_ite_test[:, 1] > ite_test)))
            coverage_exact_ite.append(np.mean((int_exact_ite_test[:, 0] < ite_test) &  (int_exact_ite_test[:, 1] > ite_test)))
            coverage_inexact_ite.append(np.mean((int_inexact_ite_test[:, 0] < ite_test) &  (int_inexact_ite_test[:, 1] > ite_test)))

            int_width_y0.append(np.diff(int_y0_test).mean())
            int_width_y1.append(np.diff(int_y1_test).mean())
            int_width_pseudo_MC_T.append(np.diff(int_pseudo_MC_ite_test).mean())
            int_width_MC_T.append(np.diff(int_MC_ite_test).mean())
            int_width_pseudo_MC_S.append(np.diff(int_pseudo_MC_S_ite_test).mean())
            int_width_MC_S.append(np.diff(int_MC_S_ite_test).mean())
            int_width_pseudo_MC_X.append(np.diff(int_pseudo_MC_X_ite_test).mean())
            int_width_MC_X.append(np.diff(int_MC_X_ite_test).mean())
            int_width_CM.append(np.diff(int_CM_ite_test).mean())
            int_width_naive_ite.append(np.diff(int_naive_ite_test).mean())
            int_width_exact_ite.append(np.diff(int_exact_ite_test).mean())
            int_width_inexact_ite.append(np.diff(int_inexact_ite_test).mean())

            rmse_y0.append(np.sqrt(np.mean((y0_test - pred_results_y0[-1])**2)))
            rmse_y1.append(np.sqrt(np.mean((y1_test - pred_results_y1[-1])**2)))
            rmse_pseudo_MC_T_ite.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_T_ite[-1])**2)))
            rmse_MC_T_ite.append(np.sqrt(np.mean((ite_test - pred_results_MC_T_ite[-1])**2)))
            rmse_pseudo_MC_S_ite.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_S_ite[-1])**2)))
            rmse_MC_S_ite.append(np.sqrt(np.mean((ite_test - pred_results_MC_S_ite[-1])**2)))
            rmse_pseudo_MC_X_ite.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_X_ite[-1])**2)))
            rmse_MC_X_ite.append(np.sqrt(np.mean((ite_test - pred_results_MC_X_ite[-1])**2)))
            rmse_CM_ite.append(np.sqrt(np.mean((ite_test - pred_results_CM_ite[-1])**2)))
            rmse_naive_ite.append(np.sqrt(np.mean((ite_test - pred_results_naive_ite[-1])**2)))
            rmse_exact_ite.append(np.sqrt(np.mean((ite_test - pred_results_exact_ite[-1])**2)))
            rmse_inexact_ite.append(np.sqrt(np.mean((ite_test - pred_results_inexact_ite[-1])**2)))

            # conformal_mean
            rmse_pseudo_MC_T_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_T_ite_conformal_mean[-1])**2)))
            rmse_MC_T_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_MC_T_ite_conformal_mean[-1])**2)))
            rmse_pseudo_MC_S_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_S_ite_conformal_mean[-1])**2)))
            rmse_MC_S_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_MC_S_ite_conformal_mean[-1])**2)))
            rmse_pseudo_MC_X_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_pseudo_MC_X_ite_conformal_mean[-1])**2)))
            rmse_MC_X_ite_conformal_mean.append(np.sqrt(np.mean((ite_test - pred_results_MC_X_ite_conformal_mean[-1])**2)))


        for i in range(len(alphas)):
            df_pred_i = pd.DataFrame({
                "int_results_y0_l": int_results_y0[i][:, 0],
                "int_results_y0_u": int_results_y0[i][:, 1],
                "int_results_y1_l": int_results_y1[i][:, 0],
                "int_results_y1_u": int_results_y1[i][:, 1],
                "int_results_pseudo_MC_T_ite_l": int_results_pseudo_MC_T_ite[i][:, 0],
                "int_results_pseudo_MC_T_ite_u": int_results_pseudo_MC_T_ite[i][:, 1],
                "int_results_MC_T_ite_l": int_results_MC_T_ite[i][:, 0],
                "int_results_MC_T_ite_u": int_results_MC_T_ite[i][:, 1],
                "int_results_pseudo_MC_S_ite_l": int_results_pseudo_MC_S_ite[i][:, 0],
                "int_results_pseudo_MC_S_ite_u": int_results_pseudo_MC_S_ite[i][:, 1],
                "int_results_MC_S_ite_l": int_results_MC_S_ite[i][:, 0],
                "int_results_MC_S_ite_u": int_results_MC_S_ite[i][:, 1],
                "int_results_pseudo_MC_X_ite_l": int_results_pseudo_MC_X_ite[i][:, 0],
                "int_results_pseudo_MC_X_ite_u": int_results_pseudo_MC_X_ite[i][:, 1],
                "int_results_MC_X_ite_l": int_results_MC_X_ite[i][:, 0],
                "int_results_MC_X_ite_u": int_results_MC_X_ite[i][:, 1],
                "int_results_CM_ite_l": int_results_CM_ite[i][:, 0],
                "int_results_CM_ite_u": int_results_CM_ite[i][:, 1],
                "int_results_naive_ite_l": int_results_naive_ite[i][:, 0],
                "int_results_naive_ite_u": int_results_naive_ite[i][:, 1],
                "int_results_exact_ite_l": int_results_exact_ite[i][:, 0],
                "int_results_exact_ite_u": int_results_exact_ite[i][:, 1],
                "int_results_inexact_ite_l": int_results_inexact_ite[i][:, 0],
                "int_results_inexact_ite_u": int_results_inexact_ite[i][:, 1],
                "pred_results_y0": pred_results_y0[i],
                "pred_results_y1": pred_results_y1[i],
                "pred_results_pseudo_MC_T_ite": pred_results_pseudo_MC_T_ite[i],
                "pred_results_MC_T_ite": pred_results_MC_T_ite[i],
                "pred_results_pseudo_MC_S_ite": pred_results_pseudo_MC_S_ite[i],
                "pred_results_MC_S_ite": pred_results_MC_S_ite[i],
                "pred_results_pseudo_MC_X_ite": pred_results_pseudo_MC_X_ite[i],
                "pred_results_MC_X_ite": pred_results_MC_X_ite[i],
                "pred_results_CM_ite": pred_results_CM_ite[i],
                "pred_results_naive_ite": pred_results_naive_ite[i],
                "pred_results_exact_ite": pred_results_exact_ite[i],
                "pred_results_inexact_ite": pred_results_inexact_ite[i],
                "pred_results_pseudo_MC_T_ite_conformal_mean": pred_results_pseudo_MC_T_ite_conformal_mean[i],
                "pred_results_MC_T_ite_conformal_mean": pred_results_MC_T_ite_conformal_mean[i],
                "pred_results_pseudo_MC_S_ite_conformal_mean": pred_results_pseudo_MC_S_ite_conformal_mean[i],
                "pred_results_MC_S_ite_conformal_mean": pred_results_MC_S_ite_conformal_mean[i],
                "pred_results_pseudo_MC_X_ite_conformal_mean": pred_results_pseudo_MC_X_ite_conformal_mean[i],
                "pred_results_MC_X_ite_conformal_mean": pred_results_MC_X_ite_conformal_mean[i],
                "test_idx": ds_test.index,
                "alpha": np.repeat(alphas[i],
                                    len(pred_results_y0[i]))
            })
            if i == 0:
                df_pred = df_pred_i
            else:
                df_pred = pd.concat([df_pred, df_pred_i], axis=0)
        if heteroscedastic_epsilon:
            df_pred.to_csv(f"../../results/outputs/r_learner/setup{setup_name}/preds/simulations_{setup_name}_{str(n)}_{learner_name}_{adaptive_conformal_name}_heteroscedastic_pred.csv", index=False)
        else:
            df_pred.to_csv(f"../../results/outputs/r_learner/setup{setup_name}/preds/simulations_{setup_name}_{str(n)}_{learner_name}_{adaptive_conformal_name}_pred.csv", index=False)

        df_eval = pd.DataFrame({
            "coverage_y0": coverage_y0,
            "coverage_y1": coverage_y1,
            "coverage_pseudo_MC_T_ite": coverage_pseudo_MC_T_ite,
            "coverage_MC_T_ite": coverage_MC_T_ite,
            "coverage_pseudo_MC_S_ite": coverage_pseudo_MC_S_ite,
            "coverage_MC_S_ite": coverage_MC_S_ite,
            "coverage_pseudo_MC_X_ite": coverage_pseudo_MC_X_ite,
            "coverage_MC_X_ite": coverage_MC_X_ite,
            "coverage_CM_ite": coverage_CM_ite,
            "coverage_naive_ite": coverage_naive_ite,
            "coverage_exact_ite": coverage_exact_ite,
            "coverage_inexact_ite": coverage_inexact_ite,
            "int_width_y0": int_width_y0,
            "int_width_y1": int_width_y1,
            "int_width_pseudo_MC_T": int_width_pseudo_MC_T,
            "int_width_MC_T": int_width_MC_T,
            "int_width_pseudo_MC_S": int_width_pseudo_MC_S,
            "int_width_MC_S": int_width_MC_S,
            "int_width_pseudo_MC_X": int_width_pseudo_MC_X,
            "int_width_MC_X": int_width_MC_X,
            "int_width_CM": int_width_CM,
            "int_width_naive_ite": int_width_naive_ite,
            "int_width_exact_ite": int_width_exact_ite,
            "int_width_inexact_ite": int_width_inexact_ite,
            "rmse_y0": rmse_y0,
            "rmse_y1": rmse_y1,
            "rmse_pseudo_MC_T_ite": rmse_pseudo_MC_T_ite,
            "rmse_MC_T_ite": rmse_MC_T_ite,
            "rmse_pseudo_MC_S_ite": rmse_pseudo_MC_S_ite,
            "rmse_MC_S_ite": rmse_MC_S_ite,
            "rmse_pseudo_MC_X_ite": rmse_pseudo_MC_X_ite,
            "rmse_MC_X_ite": rmse_MC_X_ite,
            "rmse_CM_ite": rmse_CM_ite,
            "rmse_naive_ite": rmse_naive_ite,
            "rmse_exact_ite": rmse_exact_ite,
            "rmse_inexact_ite": rmse_inexact_ite,
            "rmse_pseudo_MC_T_ite_conformal_mean": rmse_pseudo_MC_T_ite_conformal_mean,
            "rmse_MC_T_ite_conformal_mean": rmse_MC_T_ite_conformal_mean,
            "rmse_pseudo_MC_S_ite_conformal_mean": rmse_pseudo_MC_S_ite_conformal_mean,
            "rmse_MC_S_ite_conformal_mean": rmse_MC_S_ite_conformal_mean,
            "rmse_pseudo_MC_X_ite_conformal_mean": rmse_pseudo_MC_X_ite_conformal_mean,
            "rmse_MC_X_ite_conformal_mean": rmse_MC_X_ite_conformal_mean,
            "alpha": alphas
        })
        if heteroscedastic_epsilon:
            df_eval.to_csv(f"../../results/outputs/r_learner/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{adaptive_conformal_name}_heteroscedastic_eval.csv", index=False)
        else:
            df_eval.to_csv(f"../../results/outputs/r_learner/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{adaptive_conformal_name}_eval.csv", index=False)

100%|██████████| 100/100 [2:36:46<00:00, 94.07s/it] 
